In [1]:
from pathlib import Path
import requests

def download_one_file_of_raw_data(year: int, month: int) -> Path:
    # https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page
    URL = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet'
    response = requests.get(URL)

    if response.status_code ==200:
        path= f'../data/raw/rides_{year}_{month:02d}.parquet'
        open(path,"wb").write(response.content)
        return path
    else:
        raise Exception(f'{URL} is not available')

In [5]:
year=2023
month=1
download_one_file_of_raw_data(year,month)

'../data/raw/rides_2023_01.parquet'

In [6]:
import pandas as pd

rides_df = pd.read_parquet('../data/raw/rides_2023_01.parquet')
rides = rides_df[['tpep_pickup_datetime', 'PULocationID']]

#Rename Columns
rides.rename(columns={
    'tpep_pickup_datetime': 'pickup_datetime',
    'PULocationID': 'pickup_location_id',
}, inplace=True)

rides.head(10)

/var/folders/7t/f9kzz9ps7550yjz7_s6y78240000gq/T/ipykernel_4301/3061728805.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rides.rename(columns={


,pickup_datetime,pickup_location_id
0,2023-01-01 00:32:10,161
1,2023-01-01 00:55:08,43
2,2023-01-01 00:25:04,48
3,2023-01-01 00:03:48,138
4,2023-01-01 00:10:29,107
5,2023-01-01 00:50:34,161
6,2023-01-01 00:09:22,239
7,2023-01-01 00:27:12,142
8,2023-01-01 00:21:44,164
9,2023-01-01 00:39:42,141


In [7]:
rides['pickup_datetime'].describe()

count                          3066766
mean     2023-01-17 00:22:26.288164096
min                2008-12-31 23:01:42
25%      2023-01-09 16:21:57.249999872
50%         2023-01-17 08:42:29.500000
75%                2023-01-24 16:26:27
max                2023-02-01 00:56:53
Name: pickup_datetime, dtype: object

We can see that there are some dates not between Jan 01, 2023 - Jan 31, 2023 (min data is 2008)

In [ ]:
rides = rides[rides.pickup_datetime >= '2023-01-01']
rides = rides[rides.pickup_datetime < '2023-02-01']
rides['pickup_datetime'].describe()

count                          3066718
mean     2023-01-17 00:27:48.391111680
min                2023-01-01 00:00:00
25%      2023-01-09 16:22:12.249999872
50%         2023-01-17 08:42:40.500000
75%                2023-01-24 16:26:28
max                2023-01-31 23:59:59
Name: pickup_datetime, dtype: object

In [8]:
# Save validated file to transformed data file

rides.to_parquet(f'../data/transformed/validated_rides_{year}_{month:02d}.parquet')
